In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Defining fucntions
We can define a function to extract Product title. This involves getting the outer Tag Object, inner navigatable string object and title as a string value.Creating these functions will make it easy for us by just calling them when ever needed during our scraping.

In [7]:
# function to extract product title
def get_title(soup):
    try:
        #Outer tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        #Inner NavigatableString Object
        title_value = title.text
        #Title as String Value
        title_string = title_value.strip()
    
    except AttributeError:
        title_string = ""
    return title_string


# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available
 
        
        
        

In [12]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.42', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=ps5+console&rh=n%3A976460031%2Cp_36%3A1741396031&dc&ds=v1%3Axafe%2FgyRPuL4T2kmwIp6CX%2B0JadN6DlQKyDerdH8dLU&content-id=amzn1.sym.c16802f7-e9f2-4a7a-ace2-b13748bc8783%3Aamzn1.sym.c16802f7-e9f2-4a7a-ace2-b13748bc8783&pd_rd_r=c8b37f64-54c3-471c-996c-cd3303d6eb80&pd_rd_w=4ODOt&pd_rd_wg=5mbVF&pf_rd_p=c16802f7-e9f2-4a7a-ace2-b13748bc8783&pf_rd_r=3RNSR7XSYP40J8P2DWBR&qid=1668370027&rnid=1741393031&ref=sr_nr_p_36_3"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [13]:
amazon_df

,title,price,rating,reviews,availability
7,ZORBES® Dust Cover for SONY PS5 Console - Dust...,,4.4 out of 5 stars,5 ratings,Currently unavailable.
